In [1]:
import os
import re
import numpy as np
import pandas as pd
from glob import glob
from scipy import signal
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import pickle

In [ ]:

def extract_dft_features(signal_data, n_fft=2048):
    """
    Extract DFT magnitude spectrum features as described in the research paper.
    
    Parameters:
        signal_data: Input signal data
        n_fft: Number of points for FFT (default: 2048 as used in the paper)
    
    Returns:
        Magnitude spectrum features
    """
   
    dft = np.fft.fft(signal_data, n=n_fft)
    
   
    magnitude_spectrum = np.abs(dft[:n_fft//2])
    
    
    if np.max(magnitude_spectrum) > 0:
        magnitude_spectrum = magnitude_spectrum / np.max(magnitude_spectrum)
    
    return magnitude_spectrum


# def moving_average_filter(data, window_size=5):
#     """
#     Applies a moving average filter to smooth the input data.
#     """
#     return np.convolve(data, np.ones(window_size)/window_size, mode='valid')

In [4]:
def load_segment_pair_files(folder_path):
    """
    Groups CSV files in the given folder into segments by pairing the low band (L)
    and high band (H) parts.
    """
    segment_dict = {}
    pattern = re.compile(r"(\d+)([LH])_(\d+)\.csv")
    files = glob(os.path.join(folder_path, "*.csv"))
    
    for file in files:
        basename = os.path.basename(file)
        match = pattern.match(basename)
        if match:
            seg_id = match.group(1)
            band = match.group(2)   # 'L' or 'H'
            pair_idx = match.group(3)
            key = (seg_id, pair_idx)
            if key not in segment_dict:
                segment_dict[key] = {}
            segment_dict[key][band] = file
    
    segments = []
    for key, parts in segment_dict.items():
        if 'L' in parts and 'H' in parts:
            try:
                data_low = np.loadtxt(parts['L'], delimiter=',')
                data_high = np.loadtxt(parts['H'], delimiter=',')
                segments.append((data_low, data_high, parts['L'], parts['H']))
                print(f"Loaded pair: {os.path.basename(parts['L'])} and {os.path.basename(parts['H'])}")
            except Exception as e:
                print(f"Error loading segment {key}: {e}")
    return segments


In [ ]:


def extract_all_features(signal_low, signal_high, fs):
    """
    Extract features strictly following the research paper:
    use only the lower band DFT magnitude spectrum.
    
    Parameters:
        signal_low: Low band signal data
        signal_high: High band signal data (ignored)
        fs: Sampling rate (not used in this extraction)
    
    Returns:
        Feature vector of length 1024 (one-sided DFT magnitude spectrum)
    """
    # filtered_low = moving_average_filter(signal_low)
    dft_features_low = extract_dft_features(signal_low)
    return dft_features_low

In [ ]:
def add_csv_data_to_training(csv_path, X, y, class_label=0, fs=40e6, t_seg=20):
    """
    Add data from a CSV file to the training dataset.
    
    Parameters:
        csv_path: Path to the CSV file with magnitude data
        X: Feature vectors (list or numpy array)
        y: Labels (list or numpy array)
        class_label: Label to assign to this data (0 for background/non-drone, 1 for drone)
        fs: Sampling rate in Hz
        t_seg: Time segment in ms
    
    Returns:
        Updated X and y as numpy arrays
    """
    print(f"Loading data from: {csv_path}")
    
    try:
        # Convert X and y to lists if they are numpy arrays
        X_list = X.tolist() if isinstance(X, np.ndarray) else list(X)
        y_list = y.tolist() if isinstance(y, np.ndarray) else list(y)
        
        # Read the CSV file
        df = pd.read_csv(csv_path)
        
        # Extract magnitude data
        magnitude_data = df['Magnitude'].values
        
        # Calculate segment size
        samples_per_band = int(t_seg / 1000 * fs)
        segment_length = 2 * samples_per_band
        
        # Calculate number of complete segments
        total_samples = len(magnitude_data)
        num_segments = total_samples // segment_length
        
        print(f"Found {num_segments} complete segments in CSV file")
        
       
        for i in range(num_segments):
            segment = magnitude_data[i * segment_length : (i + 1) * segment_length]
            
       
            low_band = segment[:samples_per_band]
            high_band = segment[samples_per_band:]
            
         
            feature_vector = extract_all_features(low_band, high_band, fs)
            
            # Add to training data
            X_list.append(feature_vector)
            y_list.append(class_label)
        
        print(f"Added {num_segments} segments with class label {class_label} to training data")
        

        return np.array(X_list), np.array(y_list)
    
    except Exception as e:
        print(f"Error processing CSV file: {e}")
        return X, y

In [7]:

drone_folders = [
    'Data\\DroneRF\\Bepop drone',
    'Data\\DroneRF\\Phantom drone',
    'Data\\DroneRF\\AR drone'
]
background_folder = 'Data\DroneRF\Background RF activites'


In [8]:
fs = 40e6  # Sampling rate: 40 MHz


X = []
y = []


In [11]:


for folder in drone_folders:
    print(f"Loading drone segments from {folder}")
    segment_pairs = load_segment_pair_files(folder)
    for (sig_low, sig_high, file_low, file_high) in segment_pairs:
        features = extract_all_features(sig_low, sig_high, fs)
        X.append(features)
        y.append(1)


print(f"Loading background segments from {background_folder}")
background_segment_pairs = load_segment_pair_files(background_folder)
for (sig_low, sig_high, file_low, file_high) in background_segment_pairs:
    features = extract_all_features(sig_low, sig_high, fs)
    X.append(features)
    y.append(0)
non_drone_csv_path = r'Data\pluto_sdr_iq_data_20250320_122718.csv'
X, y = add_csv_data_to_training(non_drone_csv_path, X, y, class_label=0, fs=fs, t_seg=t_seg)

Loading drone segments from Data\DroneRF\Bepop drone
Loaded pair: 10000L_0.csv and 10000H_0.csv
Loaded pair: 10000L_1.csv and 10000H_1.csv
Loaded pair: 10000L_10.csv and 10000H_10.csv
Loaded pair: 10000L_11.csv and 10000H_11.csv
Loaded pair: 10000L_12.csv and 10000H_12.csv
Loaded pair: 10000L_13.csv and 10000H_13.csv
Loaded pair: 10000L_14.csv and 10000H_14.csv
Loaded pair: 10000L_15.csv and 10000H_15.csv
Loaded pair: 10000L_16.csv and 10000H_16.csv
Loaded pair: 10000L_17.csv and 10000H_17.csv
Loaded pair: 10000L_18.csv and 10000H_18.csv
Loaded pair: 10000L_19.csv and 10000H_19.csv
Loaded pair: 10000L_2.csv and 10000H_2.csv
Loaded pair: 10000L_20.csv and 10000H_20.csv
Loaded pair: 10000L_3.csv and 10000H_3.csv
Loaded pair: 10000L_4.csv and 10000H_4.csv
Loaded pair: 10000L_5.csv and 10000H_5.csv
Loaded pair: 10000L_6.csv and 10000H_6.csv
Loaded pair: 10000L_7.csv and 10000H_7.csv
Loaded pair: 10000L_8.csv and 10000H_8.csv
Loaded pair: 10000L_9.csv and 10000H_9.csv
Loaded pair: 10001L_0.

NameError: name 't_seg' is not defined

In [22]:
X, y = add_csv_data_to_training(non_drone_csv_path, X, y, class_label=0, fs=fs, t_seg=20)

Loading data from: Data\pluto_sdr_iq_data_20250320_122718.csv
Found 37 complete segments in CSV file
Added 37 segments with class label 0 to training data


In [23]:
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [24]:
X = np.array(X)
y = np.array(y)

print("Feature matrix shape:", X.shape)
print("Labels shape:", y.shape)

Feature matrix shape: (264, 1024)
Labels shape: (264,)


In [25]:
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [26]:
X

array([[0.60006784, 0.30210175, 0.52592163, ..., 0.25788792, 0.06525057,
        0.28750348],
       [0.25488565, 0.29858111, 0.25247625, ..., 0.13910876, 0.32933118,
        0.49817144],
       [0.39971281, 0.29168567, 0.14245768, ..., 0.44475691, 0.05480315,
        0.41836384],
       ...,
       [1.        , 0.0985408 , 0.03899714, ..., 0.03353172, 0.01515668,
        0.03891594],
       [1.        , 0.02726214, 0.00406939, ..., 0.01674093, 0.03206413,
        0.02320978],
       [1.        , 0.07862266, 0.0201619 , ..., 0.01455576, 0.01978327,
        0.01836937]], shape=(264, 1024))

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)



In [28]:
y_train_resampled

array([1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [39]:
y_train_resampled.shape

(298,)

In [ ]:
import pandas as pd


columns = X_train.columns if hasattr(X_train, 'columns') else [f"feature_{i}" for i in range(X_train.shape[1])]


df_train_resampled = pd.DataFrame(X_train_resampled, columns=columns)
df_train_resampled['label'] = y_train_resampled 

# Save to CSV
df_train_resampled.to_csv('train_resampled.csv', index=False)
print("Resampled training data saved to 'train_resampled.csv'")

Resampled training data saved to 'train_resampled.csv'


In [40]:
df_train_resampled.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_1015,feature_1016,feature_1017,feature_1018,feature_1019,feature_1020,feature_1021,feature_1022,feature_1023,label
0,0.766627,0.578235,0.157086,0.568760,0.768131,0.428041,0.041847,0.045593,0.209855,0.029573,...,0.156485,0.119663,0.522185,0.143958,0.172600,0.394562,0.176607,0.352638,0.292205,1
1,0.129720,0.398150,0.253483,0.282366,0.308898,0.064255,0.327565,0.282170,0.331814,0.356929,...,0.034543,0.367078,0.091329,0.170362,0.130283,0.233197,0.160886,0.179582,0.056798,1
2,0.627913,0.493194,0.091230,0.131886,0.443734,0.135997,0.603839,0.154673,0.185795,0.102805,...,0.416081,0.165423,0.148593,0.160592,0.084939,0.144349,0.158086,0.426031,0.217403,1
3,1.000000,0.038643,0.009771,0.030809,0.022795,0.033609,0.042167,0.027967,0.021877,0.038506,...,0.018463,0.022437,0.016195,0.005150,0.011961,0.024889,0.004297,0.015382,0.030027,0
4,0.284546,0.454379,0.146484,0.149473,0.269185,0.225193,0.066250,0.405332,0.226789,0.390357,...,0.458612,0.193191,0.435854,0.158643,0.274269,0.141692,0.291278,0.131933,0.214344,1


In [ ]:

df_test = pd.DataFrame(X_test, columns=columns)
df_test['label'] = y_test

# Save to CSV
df_test.to_csv('test.csv', index=False)
print("Test data saved to 'test.csv'")

Test data saved to 'test.csv'


In [ ]:




# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)



xgb_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'eta': 0.1,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_weight': 1,
    'gamma': 0,
    'n_estimators': 100,
    'random_state': 42
}


print("Performing 10-fold cross-validation...")
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
xgb_model = xgb.XGBClassifier(**xgb_params)

cv_scores = cross_val_score(xgb_model, X_train_resampled, y_train_resampled, cv=kfold, scoring='accuracy')
print(f"Cross-validation accuracy: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")

Performing 10-fold cross-validation...
Cross-validation accuracy: 0.9429 (±0.0541)


In [31]:

xgb_model.fit(X_train_resampled, y_train_resampled)

# Save the trained model using pickle or XGBoost's own save_model method
with open('xgb_model_scaled.pkl', 'wb') as f:
    pickle.dump(xgb_model, f)


In [32]:
y_pred = xgb_model.predict(X_test)
y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)


Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.44      0.58        16
           1       0.80      0.97      0.88        37

    accuracy                           0.81        53
   macro avg       0.84      0.71      0.73        53
weighted avg       0.82      0.81      0.79        53


Confusion Matrix:
[[ 7  9]
 [ 1 36]]


In [114]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, StratifiedKFold

# Define SVM parameters (adjust based on your problem)
svm_params = {
    'kernel': 'linear',       # Common kernels: 'linear', 'rbf', 'poly'
    'C': 1.0,              # Regularization parameter (higher = stricter margin)
    'gamma': 'scale',      # Kernel coefficient ('scale' or 'auto')
    'random_state': 42,
    'probability': True    # Enable probability estimates (for ROC-AUC)
}

# Initialize SVM model
svm_model = SVC(**svm_params)

# Cross-validation using 10-fold
print("Performing 10-fold cross-validation...")
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

cv_scores = cross_val_score(
    svm_model, 
    X_train_resampled, 
    y_train_resampled, 
    cv=kfold, 
    scoring='accuracy', 
    n_jobs=-1  # Use all CPU cores
)

print(f"Cross-validation accuracy: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")

Performing 10-fold cross-validation...
Cross-validation accuracy: 0.8890 (±0.0667)


In [115]:

svm_model.fit(X_train_resampled, y_train_resampled)

# Save the trained model using pickle or XGBoost's own save_model method
with open('svm_model_scaled.pkl', 'wb') as f:
    pickle.dump(svm_model, f)


In [116]:
y_pred = svm_model.predict(X_test)
y_pred_proba = svm_model.predict_proba(X_test)[:, 1]

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)


Classification Report:
              precision    recall  f1-score   support

           0       0.12      0.25      0.17         4
           1       0.92      0.83      0.88        42

    accuracy                           0.78        46
   macro avg       0.52      0.54      0.52        46
weighted avg       0.85      0.78      0.81        46


Confusion Matrix:
[[ 1  3]
 [ 7 35]]


In [44]:
kfold

StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

In [35]:
# Hyperparameter tuning
param_grid = {
    'eta': [0.01, 0.05, 0.1],
    'max_depth': [3, 6, 9],
    'n_estimators': [50, 100, 200],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'min_child_weight': [1, 3, 5]
}

grid_search = GridSearchCV(
    estimator=xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss', random_state=42),
    param_grid=param_grid,
    cv=kfold,
    scoring='accuracy',
    verbose=2,
    n_jobs=-1
)

print("Starting grid search for optimal parameters...")
grid_search.fit(X_train_scaled, y_train)

print("\nBest parameters found:")
print(grid_search.best_params_)
print(f"Best cross-validation accuracy: {grid_search.best_score_:.4f}")

# Train final model with best parameters
best_model = grid_search.best_estimator_
best_model.fit(X_train_scaled, y_train)

Starting grid search for optimal parameters...
Fitting 10 folds for each of 729 candidates, totalling 7290 fits


KeyboardInterrupt: 

In [ ]:
# # Evaluate on test set
# y_pred = best_model.predict(X_test_scaled)
# y_pred_proba = best_model.predict_proba(X_test_scaled)[:, 1]

# print("\nClassification Report:")
# print(classification_report(y_test, y_pred))

# # Calculate confusion matrix
# cm = confusion_matrix(y_test, y_pred)
# print("\nConfusion Matrix:")
# print(cm)


Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        38

    accuracy                           1.00        46
   macro avg       1.00      1.00      1.00        46
weighted avg       1.00      1.00      1.00        46


Confusion Matrix:
[[ 8  0]
 [ 0 38]]


In [ ]:
# Calculate ROC curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)
print(f"ROC AUC: {roc_auc:.4f}")

ROC AUC: 1.0000


In [ ]:

feature_importance = xgb_model.feature_importances_
sorted_idx = np.argsort(feature_importance)[::-1]

print("\nTop 20 Most Important Features:")
for i in range(min(20, len(sorted_idx))):
    print(f"Feature {sorted_idx[i]}: {feature_importance[sorted_idx[i]]:.4f}")


Top 20 Most Important Features:
Feature 78: 0.0282
Feature 317: 0.0248
Feature 651: 0.0236
Feature 66: 0.0187
Feature 537: 0.0183
Feature 816: 0.0182
Feature 510: 0.0168
Feature 0: 0.0155
Feature 177: 0.0151
Feature 708: 0.0146
Feature 640: 0.0145
Feature 992: 0.0140
Feature 366: 0.0134
Feature 471: 0.0134
Feature 472: 0.0127
Feature 690: 0.0120
Feature 801: 0.0116
Feature 638: 0.0109
Feature 566: 0.0108
Feature 171: 0.0107


In [ ]:
import pickle
with open('drone_detector_xgboost_smote_model.pkl', 'wb') as model_file:
    pickle.dump(best_model, model_file)

## Test on SDR csv

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from scipy import signal

# def moving_average_filter(data, window_size=5):
#     """
#     Applies a moving average filter (FIR filter with uniform weights)
#     to smooth the input data.
#     """
#     return np.convolve(data, np.ones(window_size)/window_size, mode='valid')

def extract_dft_features(signal_data, n_fft=2048):
    """
    Extract DFT magnitude spectrum features as described in the research paper.
    
    Parameters:
        signal_data: Input signal data
        n_fft: Number of points for FFT (default: 2048 as used in the paper)
    
    Returns:
        Magnitude spectrum features
    """
    # Compute DFT
    dft = np.fft.fft(signal_data, n=n_fft)
    
    # Extract one-sided magnitude spectrum (first half of the spectrum)
    magnitude_spectrum = np.abs(dft[:n_fft//2])
    
    # Normalize the magnitude spectrum
    if np.max(magnitude_spectrum) > 0:
        magnitude_spectrum = magnitude_spectrum / np.max(magnitude_spectrum)
    
    return magnitude_spectrum



def extract_all_features(signal_low, signal_high, fs):
    """
    Extract features strictly following the research paper:
    use only the lower band DFT magnitude spectrum.
    
    Parameters:
        signal_low: Low band signal data
        signal_high: High band signal data (ignored)
        fs: Sampling rate (not used in this extraction)
    
    Returns:
        Feature vector of length 1024 (one-sided DFT magnitude spectrum)
    """
    # filtered_low = moving_average_filter(signal_low)
    dft_features_low = extract_dft_features(signal_low)
    return dft_features_low

def load_model(model_path):
    """Load a trained classification model from a pickle file."""
    try:
        with open(model_path, 'rb') as f:
            model = pickle.load(f)
        print(f"Model loaded from: {model_path}")
        return model
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

def visualize_results(predictions, segment_numbers):
    """Visualize prediction results for each segment."""
    plt.figure(figsize=(12, 6))
    plt.bar(segment_numbers, predictions, color=['green' if p == 1 else 'red' for p in predictions])
    plt.axhline(y=0.5, color='black', linestyle='-', alpha=0.3)
    plt.xlabel('Segment Number')
    plt.ylabel('Prediction (1=Drone, 0=Background)')
    plt.title('Drone Detection Results by Segment')
    plt.ylim(-0.1, 1.1)
    plt.xticks(segment_numbers)
    plt.grid(True, alpha=0.3)
    for i, v in enumerate(predictions):
        plt.text(segment_numbers[i], v+0.05, str(v), ha='center')
    plt.show()

def visualize_features(feature_vector, n_dft=1024, n_stats=12, n_psd=10):
    """Visualize different parts of the feature vector."""
    # Separate different feature types
    dft_features = feature_vector[:n_dft]
    stats_features = feature_vector[n_dft:n_dft+n_stats]
    psd_features = feature_vector[n_dft+n_stats:]
    
    fig, axs = plt.subplots(3, 1, figsize=(12, 10))
    
    # Plot DFT features
    axs[0].plot(dft_features)
    axs[0].set_title('DFT Magnitude Spectrum Features')
    axs[0].set_xlabel('Frequency Bin')
    axs[0].set_ylabel('Normalized Magnitude')
    axs[0].grid(True)
    
    # Plot statistical features
    stats_labels = ['Mean', 'Std', 'Max', 'Min', 'Median', '25th', '75th', 'Var', 
                   'RMS', 'MAD', 'Above Mean', 'Rising Edges']
    axs[1].bar(stats_labels, stats_features)
    axs[1].set_title('Statistical Features')
    axs[1].set_ylabel('Value')
    plt.setp(axs[1].get_xticklabels(), rotation=45, ha='right')
    axs[1].grid(True)
    
    # Plot PSD statistical features
    psd_labels = ['Mean', 'Std', 'Max', 'Min', 'Median', '25th', '75th', 'Var', 
                 'Spectral Centroid', 'Spectral Bandwidth']
    axs[2].bar(psd_labels, psd_features)
    axs[2].set_title('PSD Statistical Features')
    axs[2].set_ylabel('Value')
    plt.setp(axs[2].get_xticklabels(), rotation=45, ha='right')
    axs[2].grid(True)
    
    plt.tight_layout()
    plt.show()

def main():
    # Parameters
    fs = 40e6  # Sampling rate: 40 MHz
    window_size = 5  # For moving average filter
    t_seg = 20  # Time segment in ms
    n_fft = 2048  # FFT size for DFT features
    nperseg = 1024  # For Welch's method
    
   
    csv_path = r'Data\pluto_sdr_iq_data_20250320_122718.csv'
    
   
    print(f"Loading data from: {csv_path}")
    df = pd.read_csv(csv_path)
    

    magnitude_data = df['Magnitude'].values
    

    total_samples = len(magnitude_data)
    samples_per_band = int(t_seg / 1000 * fs)
    segment_length = 2 * samples_per_band
    
    if total_samples < segment_length:
        print("Error: Not enough samples in the CSV file to form a single segment.")
        return
    
    
    num_segments = total_samples // segment_length
    print(f"Total segments found: {num_segments}")
    

    model_path = 'xgb_model_scaled.pkl'
    model = load_model(model_path)
    if model is None:
        return
    
    
    predictions = []
    features_list = []
    
    for i in range(num_segments):
        segment = magnitude_data[i * segment_length : (i + 1) * segment_length]
        
 
        low_band = segment[:samples_per_band]
        high_band = segment[samples_per_band:]
        
   
        feature_vector = extract_all_features(low_band, high_band, fs)
        features_list.append(feature_vector)
        
  
        feature_vector_reshaped = feature_vector.reshape(1, -1)
        pred = model.predict(feature_vector_reshaped)
        predictions.append(pred[0])
        
        pred_prob = model.predict_proba(feature_vector_reshaped)
        print(f"Segment {i+1}: Prediction = {pred[0]} (Confidence: {pred_prob[0][1]:.4f})")
    
    
    print("\nFinal Predictions for each segment:")
    print(predictions)
    print(f"Drone detected in {sum(predictions)} out of {len(predictions)} segments.")
    
    
    

    if num_segments > 0:
        
        final_decision = 1 if sum(predictions)/len(predictions) > 0.5 else 0
        print(f"\nFinal decision based on majority voting: {'DRONE DETECTED' if final_decision == 1 else 'NO DRONE DETECTED'}")

if __name__ == "__main__":
    main()

Loading data from: Data\pluto_sdr_iq_data_20250320_122718.csv
Total segments found: 37
Model loaded from: xgb_model_scaled.pkl
Segment 1: Prediction = 0 (Confidence: 0.0046)
Segment 2: Prediction = 0 (Confidence: 0.0046)
Segment 3: Prediction = 0 (Confidence: 0.0051)
Segment 4: Prediction = 0 (Confidence: 0.0056)
Segment 5: Prediction = 0 (Confidence: 0.0056)
Segment 6: Prediction = 0 (Confidence: 0.0050)
Segment 7: Prediction = 0 (Confidence: 0.0051)
Segment 8: Prediction = 0 (Confidence: 0.0046)
Segment 9: Prediction = 0 (Confidence: 0.0051)
Segment 10: Prediction = 0 (Confidence: 0.0051)
Segment 11: Prediction = 0 (Confidence: 0.0056)
Segment 12: Prediction = 0 (Confidence: 0.0061)
Segment 13: Prediction = 0 (Confidence: 0.0056)
Segment 14: Prediction = 0 (Confidence: 0.0078)
Segment 15: Prediction = 0 (Confidence: 0.0056)
Segment 16: Prediction = 0 (Confidence: 0.0056)
Segment 17: Prediction = 0 (Confidence: 0.0046)
Segment 18: Prediction = 0 (Confidence: 0.0046)
Segment 19: Predic